# Setup

In [30]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

jdbc_path = "/home/hello/.jdbc/postgresql-42.7.7.jar"

spark = SparkSession.builder \
    .appName("PagilaInNotebook") \
    .config("spark.jars", jdbc_path) \
    .getOrCreate()

load_dotenv()

url = "jdbc:postgresql://localhost:5432/pagila"

spark.conf.set("spark.sql.shuffle.partitions", "50")

In [24]:
def read_table_from_db(table_name):
    return spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", table_name) \
    .options(**{"user": os.getenv("DB_USER"), "password": os.getenv("PASSWORD"), "driver": "org.postgresql.Driver"}) \
    .load()

# Task 1

In [33]:
df_category = read_table_from_db("public.category")
df_film_category = read_table_from_db("public.film_category")

result_df = df_film_category.join(broadcast(df_category), on="category_id") \
    .groupBy("name") \
    .agg(count("name").alias("num_of_films")) \
    .orderBy(desc(col("num_of_films")))

result_df.show(truncate=False)

+-----------+------------+
|name       |num_of_films|
+-----------+------------+
|Music      |152         |
|Drama      |152         |
|Travel     |151         |
|Games      |150         |
|Foreign    |150         |
|Children   |150         |
|Action     |149         |
|Sci-Fi     |149         |
|Animation  |148         |
|New        |147         |
|Family     |147         |
|Classics   |147         |
|Sports     |145         |
|Documentary|145         |
|Comedy     |143         |
|Horror     |142         |
+-----------+------------+



# Task 2

In [68]:
from pyspark.sql.functions import sum as _sum  

df_rental = read_table_from_db("rental")
df_inventory = read_table_from_db("inventory")
df_film_actor = read_table_from_db("film_actor")
df_actor = read_table_from_db("actor")

film_count = df_rental.join(df_inventory, on="inventory_id") \
    .groupBy("film_id") \
    .count()


df_actor = df_actor.withColumn(
    "actor_name",
    concat_ws(" ", "first_name", "last_name")
)


res = df_film_actor.join(film_count, on="film_id", how="left") \
    .join(df_actor, on="actor_id") \
    .groupBy("actor_name") \
    .agg(_sum("count").alias("rent_count")) \
    .orderBy(desc(col("rent_count")))
    
res.show(10)


+------------------+----------+
|        actor_name|rent_count|
+------------------+----------+
|       SUSAN DAVIS|       825|
|    GINA DEGENERES|       753|
|    MATTHEW CARREY|       678|
|       MARY KEITEL|       674|
|ANGELA WITHERSPOON|       654|
|       WALTER TORN|       640|
|       HENRY BERRY|       612|
|       JAYNE NOLTE|       611|
|        VAL BOLGER|       605|
|     SANDRA KILMER|       604|
+------------------+----------+
only showing top 10 rows



# Task 3

In [ ]:
df_rental = read_table_from_db("rental")
df_inventory = read_table_from_db("inventory")
df_category = read_table_from_db("category")
df_film_category = read_table_from_db("film_category")
df_payment = read_table_from_db("payment")


df_film_revenue = df_payment.join(df_rental , on="rental_id") \
    .join(df_inventory, on="inventory_id") \
    .groupBy("film_id") \
    .agg(_sum("amount").alias("spent_on_film"))
    
res = df_film_category.join(df_film_revenue, on="film_id") \
    .join(df_category, on="category_id") \
    .groupBy("name") \
    .agg(_sum("spent_on_film").alias("category_revenue")) \
    .orderBy(desc(col("name")))
    
res.show(1)


+------+----------------+
|  name|category_revenue|
+------+----------------+
|Travel|         9793.98|
+------+----------------+
only showing top 1 row



# Task 4

In [ ]:
df_film = read_table_from_db("film")
df_inventory = read_table_from_db("inventory")

res = df_film.join(df_inventory, on="film_id", how="left") \
    .filter(col("inventory_id").isNotNull()).dropDuplicates(["film_id"]).select(["film_id", "title"])
    
res.show()

# Task 5

In [86]:
from pyspark.sql.window import Window

df_category = read_table_from_db("category").alias(("category"))
df_film_category = read_table_from_db("film_category").alias("film_category")
df_film_actor = read_table_from_db("film_actor").alias("film_actor")
df_actor = read_table_from_db("actor").alias("actor")

df_actor = df_actor.withColumn(
    "actor_name",
    concat_ws(" ", "first_name", "last_name")
)

df_selected_actors = df_film_actor.join(df_film_category, on="film_id") \
    .join(df_category, on="category_id") \
    .join(df_actor, on="actor_id") \
    .filter(col("category.name") == "Children") \
    .groupBy("actor_name") \
    .agg(count(col("film_actor.film_id")).alias("cnt"))
    
windowSpec = Window.orderBy(desc(col("cnt")))

top_actors = df_selected_actors.withColumn("rank", row_number().over(windowSpec)).filter(col("rank") <= 3).select("actor_name")

top_actors.show()


25/06/23 18:13:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+
|  actor_name|
+------------+
|RICHARD PENN|
|EWAN GOODING|
|SIDNEY CROWE|
+------------+



# Task 6

In [ ]:
df_city = read_table_from_db("city").alias("city")
df_address = read_table_from_db("address").alias("address")
df_customer = read_table_from_db("customer").alias("customer")

res = df_customer.join(df_address, on="address_id") \
    .join(df_city, on="city_id") \
    .groupBy(col("city.city")) \
    .agg(
        _sum(col("customer.active")).alias("active_cnt"), 
        count(col("customer.active")).alias("cnt")) \
    .withColumn("inactive_cnt", col("cnt") - col("active_cnt")) \
    .orderBy(desc(col("inactive_cnt"))) \
    .drop("cnt")
    
res.show()

+----------------+----------+------------+
|            city|active_cnt|inactive_cnt|
+----------------+----------+------------+
| Southend-on-Sea|         0|           1|
|       Najafabad|         0|           1|
|         Bat Yam|         0|           1|
|         Wroclaw|         0|           1|
|Charlotte Amalie|         0|           1|
|        Xiangfan|         0|           1|
|          Amroha|         0|           1|
|       Pingxiang|         0|           1|
|      Kumbakonam|         0|           1|
|   Coatzacoalcos|         0|           1|
|        Uluberia|         0|           1|
|     Szkesfehrvr|         0|           1|
|          Ktahya|         0|           1|
|          Kamyin|         0|           1|
|          Daxian|         0|           1|
|         Anpolis|         1|           0|
|            Fuyu|         1|           0|
|        Mannheim|         1|           0|
|           Bchar|         1|           0|
|         Zaoyang|         1|           0|
+----------

# Task 7

In [ ]:
# df_rental_summary = df_rental.join("df_inventory", on="inventory_id") \
#     .join(df_customer, on="custormer_id") \
#     .join(df_address, on="address_id") \
#     .join(df_city, on="city_id") \
#     .join(df_film_category, on="film_id") \
#     .join(df_category, on="category_id")


finish it after feedback about 7th task from sql